In [23]:
import pandas as pd
import requests

# устанавливаем даты
start_date = "2023-03-01"
end_date = "2023-09-01"
# делаем две ссылки
visit_url = (
    f"https://data-charts-api.hexlet.app/visits?begin={start_date}&end={end_date}"
)
reg_url = f"https://data-charts-api.hexlet.app/registrations?begin={start_date}&end={end_date}"
# теперь тянем визиты и регистрации
visits = requests.get(visit_url)
registrations = requests.get(reg_url)
# Получаем данные в формате JSON
visits = visits.json()
registrations = registrations.json()
# конвертирируем в датафрейм
registrations = pd.DataFrame(registrations)
visits = pd.DataFrame(visits)
# убираем ботов
filtered_visits = visits[
    ~visits["user_agent"].str.contains("bot", case=False, na=False)
]
# конвертируем в дату
filtered_visits["datetime"] = pd.to_datetime(filtered_visits["datetime"])
# конвертируем в дату
registrations["datetime"] = pd.to_datetime(registrations["datetime"])
# сортируем и берем последний визит
filtered_visits = filtered_visits.sort_values(by=["visit_id", "datetime"])
last_visit = filtered_visits.groupby("visit_id").last().reset_index()
# группируем по датам (преобразуем) и платформам
last_visit["date_group"] = last_visit["datetime"].dt.date
df_visit = (
    last_visit.groupby(["platform", "date_group"]).size().reset_index(name="visits")
)
# регистрации по датам (преобразуем) и платформам
registrations["date_group"] = registrations["datetime"].dt.date
df_reg = (
    registrations.groupby(["platform", "date_group"])
    .size()
    .reset_index(name="registrations")
)
df = pd.merge(df_visit, df_reg, on=["platform", "date_group"], how="outer").fillna(0)
df["conversion"] = 100 * df["registrations"] / df["visits"]
df = df.sort_values(by=["date_group"])
df = df[["date_group", "platform", "visits", "registrations", "conversion"]]
df.to_json("./conversion.json")


C:\Users\Asus\AppData\Local\Temp\ipykernel_24188\280541344.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_visits['datetime'] = pd.to_datetime(filtered_visits['datetime'])
